# Lesson 7: Parameterized Tests

In this lesson, you'll learn how to write one test that runs with many different inputs.

---

## How to Start These Lessons

If you're reading this in a text editor and need to open it in Jupyter:

```bash
cd /path/to/pyside_tutorial
python start_lessons.py
```

**To stop the Jupyter server:** Press `Ctrl+C` in the terminal.

---

## What You'll Learn

- The problem with repetitive tests
- Using `@pytest.mark.parametrize`
- Naming test cases with `pytest.param`
- How our project tests 71 cases efficiently

---

## The Problem: Repetitive Tests

Let's say we want to test that invalid years raise errors. We might write:

In [ ]:
import pytest

class Movie:
    def __init__(self, title, year, director, duration):
        if year < 1895:
            raise ValueError("Year must be 1895 or later")
        self.title = title
        self.year = year
        self.director = director
        self.duration = duration

# Testing multiple invalid years - lots of repetition!
def test_year_1800_too_early():
    with pytest.raises(ValueError):
        Movie('Movie', 1800, 'Director', 90)

def test_year_1894_off_by_one():
    with pytest.raises(ValueError):
        Movie('Movie', 1894, 'Director', 90)

def test_year_1700_way_too_early():
    with pytest.raises(ValueError):
        Movie('Movie', 1700, 'Director', 90)

# Run them
test_year_1800_too_early()
test_year_1894_off_by_one()
test_year_1700_way_too_early()
print("All tests passed!")

These tests are almost identical! Only the year changes. This is:

- **Tedious** - lots of copy-paste
- **Error-prone** - easy to make mistakes when copying
- **Hard to maintain** - if the test logic changes, you update many places

**Parameterized tests** solve this problem.

---

## `@pytest.mark.parametrize`

This decorator lets you run one test function with multiple sets of inputs.

Here's the basic syntax:

```python
@pytest.mark.parametrize("param_name", [value1, value2, value3])
def test_something(param_name):
    # This runs 3 times - once for each value
    ...
```

In [ ]:
import pytest

# Instead of 3 functions, we have 1 that runs 3 times
@pytest.mark.parametrize("year", [1800, 1894, 1700])
def test_invalid_year(year):
    with pytest.raises(ValueError):
        Movie('Movie', year, 'Director', 90)

# In Jupyter, we can test like this:
for year in [1800, 1894, 1700]:
    try:
        Movie('Movie', year, 'Director', 90)
        print(f"FAIL: Year {year} should have raised error")
    except ValueError:
        print(f"PASS: Year {year} correctly raised ValueError")

When pytest runs this test, it creates 3 separate test cases:
- `test_invalid_year[1800]`
- `test_invalid_year[1894]`
- `test_invalid_year[1700]`

One test definition, three test runs!

---

## Multiple Parameters

You can pass multiple values for each test case.

Here's an example from our project (`test_media_catalogue.py` lines 45-56):

In [ ]:
class Movie:
    def __init__(self, title, year, director, duration):
        if not isinstance(title, str):
            raise ValueError("Title must be a string")
        if not title.strip():
            raise ValueError("Title cannot be empty")
        if year < 1895:
            raise ValueError("Year must be 1895 or later")
        if not isinstance(director, str):
            raise ValueError("Director must be a string")
        if not director.strip():
            raise ValueError("Director cannot be empty")
        if not any(c.isalpha() for c in director):
            raise ValueError("Director must contain at least one letter")
        if duration <= 0:
            raise ValueError("Duration must be positive")
        
        self.title = title
        self.year = year
        self.director = director
        self.duration = duration

In [ ]:
# Test multiple types of validation errors
# Each tuple is (title, year, director, duration, expected_error)

test_cases = [
    ('', 2000, 'Director', 90, 'Title cannot be empty'),
    ('   ', 2000, 'Director', 90, 'Title cannot be empty'),
    ('Movie', 1800, 'Director', 90, 'Year must be 1895 or later'),
    ('Movie', 1894, 'Director', 90, 'Year must be 1895 or later'),
    ('Movie', 2000, '', 90, 'Director cannot be empty'),
    ('Movie', 2000, '   ', 90, 'Director cannot be empty'),
    ('Movie', 2000, '123', 90, 'Director must contain at least one letter'),
    ('Movie', 2000, 'Director', 0, 'Duration must be positive'),
    ('Movie', 2000, 'Director', -90, 'Duration must be positive'),
]

# Run all test cases
for title, year, director, duration, expected_error in test_cases:
    try:
        Movie(title, year, director, duration)
        print(f"FAIL: Should have raised error for {expected_error}")
    except ValueError as e:
        if str(e) == expected_error:
            print(f"PASS: Got expected error: {expected_error}")
        else:
            print(f"FAIL: Expected '{expected_error}', got '{e}'")

In pytest, this would be:

```python
@pytest.mark.parametrize("title,year,director,duration,expected_error", [
    ('', 2000, 'Director', 90, 'Title cannot be empty'),
    ('   ', 2000, 'Director', 90, 'Title cannot be empty'),
    ('Movie', 1800, 'Director', 90, 'Year must be 1895 or later'),
    # ... more cases
])
def test_movie_validation_errors(title, year, director, duration, expected_error):
    with pytest.raises(ValueError) as e:
        Movie(title, year, director, duration)
    assert str(e.value) == expected_error
```

One function, 9 test cases!

---

## Naming Test Cases with `pytest.param`

When a test fails, you want to know which case failed. By default, pytest uses the values:

```
test_invalid_year[1800]
test_invalid_year[1894]
```

But with many parameters, this gets hard to read:

```
test_movie_validation_errors[-2000-Director-90-Title cannot be empty]
```

`pytest.param` lets you give test cases meaningful names:

In [ ]:
# From our project (test_media_catalogue.py lines 45-56)
# Note the id= parameter that names each case

test_cases_with_names = [
    pytest.param('', 2000, 'Director', 90, 'Title cannot be empty', id='empty_title'),
    pytest.param('   ', 2000, 'Director', 90, 'Title cannot be empty', id='whitespace_title'),
    pytest.param('Movie', 1800, 'Director', 90, 'Year must be 1895 or later', id='year_1800_too_early'),
    pytest.param('Movie', 1894, 'Director', 90, 'Year must be 1895 or later', id='year_1894_off_by_one'),
    pytest.param('Movie', 2000, '', 90, 'Director cannot be empty', id='empty_director'),
    pytest.param('Movie', 2000, '   ', 90, 'Director cannot be empty', id='whitespace_director'),
    pytest.param('Movie', 2000, '123', 90, 'Director must contain at least one letter', id='director_only_numbers'),
    pytest.param('Movie', 2000, 'Director', 0, 'Duration must be positive', id='zero_duration'),
    pytest.param('Movie', 2000, 'Director', -90, 'Duration must be positive', id='negative_duration'),
]

print("These test cases would appear in pytest as:")
for case in test_cases_with_names:
    print(f"  test_movie_validation_errors[{case.id}]")

Now when a test fails, you see:

```
FAILED test_media_catalogue.py::TestMovie::test_movie_validation_errors[empty_title]
```

Much clearer than a bunch of values!

---

## The Actual Test from Our Project

Here's exactly how we test validation in our project (`test_media_catalogue.py` lines 45-62):

```python
@pytest.mark.parametrize("title,year,director,duration,expected_error", [
    pytest.param('', 2000, 'Director', 90, 'Title cannot be empty', id='empty_title'),
    pytest.param('   ', 2000, 'Director', 90, 'Title cannot be empty', id='whitespace_title'),
    pytest.param('Movie', 1800, 'Director', 90, 'Year must be 1895 or later', id='year_1800_too_early'),
    pytest.param('Movie', 1894, 'Director', 90, 'Year must be 1895 or later', id='year_1894_off_by_one'),
    pytest.param('Movie', 2000, '', 90, 'Director cannot be empty', id='empty_director'),
    pytest.param('Movie', 2000, '   ', 90, 'Director cannot be empty', id='whitespace_director'),
    pytest.param('Movie', 2000, '123', 90, 'Director must contain at least one letter', id='director_only_numbers'),
    pytest.param('Movie', 2000, '42', 90, 'Director must contain at least one letter', id='director_just_number'),
    pytest.param('Movie', 2000, 'Director', 0, 'Duration must be positive', id='zero_duration'),
    pytest.param('Movie', 2000, 'Director', -90, 'Duration must be positive', id='negative_duration'),
])
def test_movie_validation_errors(self, title, year, director, duration, expected_error):
    """Test that invalid inputs raise appropriate ValueError."""
    with pytest.raises(ValueError) as e:
        Movie(title, year, director, duration)

    assert str(e.value) == expected_error
```

10 test cases from 1 test function!

---

## Boundary Testing

Parameterized tests are perfect for testing edge cases. From our project (`test_media_catalogue.py` lines 27-41):

```python
@pytest.mark.parametrize("title,year,director,duration", [
    pytest.param('First Film', 1895, 'Lumiere', 1, id='year_1895_first_valid'),
    pytest.param('Short', 2000, 'Director', 1, id='duration_1_minimum'),
    pytest.param('Future', 2099, 'Director', 120, id='year_2099_future'),
    pytest.param('A' * 500, 2000, 'Director', 90, id='very_long_title'),
    pytest.param('1917', 2019, 'Sam Mendes', 119, id='numeric_title_1917'),
    pytest.param('300', 2006, 'Zack Snyder', 117, id='numeric_title_300'),
    pytest.param('42', 2013, 'Brian Helgeland', 128, id='numeric_title_42'),
])
def test_valid_movie_boundary_cases(self, title, year, director, duration):
    """Test that boundary values create valid movies."""
    movie = Movie(title, year, director, duration)
    assert movie.title == title
    assert movie.year == year
```

This tests:
- Year 1895 (first year movies existed)
- Duration of 1 minute (minimum valid)
- Far future years
- Very long titles
- Numeric titles (like the movie "1917")

---

## Type Validation Tests

Here's another example from our project (`test_media_catalogue.py` lines 66-77):

```python
@pytest.mark.parametrize("title,expected_error", [
    pytest.param(123, 'Title must be a string', id='title_int'),
    pytest.param(['The', 'Matrix'], 'Title must be a string', id='title_list'),
    pytest.param({'name': 'Movie'}, 'Title must be a string', id='title_dict'),
    pytest.param(None, 'Title must be a string', id='title_none'),
])
def test_movie_title_type_validation(self, title, expected_error):
    """Test that non-string titles raise ValueError."""
    with pytest.raises(ValueError) as e:
        Movie(title, 2000, 'Director', 90)
    assert str(e.value) == expected_error
```

This tests what happens when you pass the wrong type:
- A number instead of a string
- A list instead of a string
- A dictionary instead of a string
- `None` instead of a string

---

## Try It Yourself

### Exercise 1: Convert to Parameterized Test

Convert these repetitive tests into one parameterized test:

In [ ]:
# Exercise 1: These tests are repetitive
# Convert them to use parameterization

def test_even_2():
    assert 2 % 2 == 0

def test_even_4():
    assert 4 % 2 == 0

def test_even_6():
    assert 6 % 2 == 0

def test_even_100():
    assert 100 % 2 == 0

# Your parameterized version here:
# @pytest.mark.parametrize(...)
# def test_even_numbers(...):
#     ...

<details>
<summary><b>Show Answer</b></summary>

```python
@pytest.mark.parametrize("number", [2, 4, 6, 100])
def test_even_numbers(number):
    assert number % 2 == 0
```

Or with named test cases:

```python
@pytest.mark.parametrize("number", [
    pytest.param(2, id='two'),
    pytest.param(4, id='four'),
    pytest.param(6, id='six'),
    pytest.param(100, id='hundred'),
])
def test_even_numbers(number):
    assert number % 2 == 0
```

</details>

### Exercise 2: Multiple Parameters

Write a parameterized test for this function that checks multiple input/output pairs:

In [ ]:
# Exercise 2: Test this function with multiple inputs

def double(n):
    return n * 2

# Test cases to verify:
# double(1) should return 2
# double(5) should return 10
# double(0) should return 0
# double(-3) should return -6

# Your parameterized test:
# @pytest.mark.parametrize("input_val,expected", [...])
# def test_double(...):
#     ...

<details>
<summary><b>Show Answer</b></summary>

```python
@pytest.mark.parametrize("input_val,expected", [
    pytest.param(1, 2, id='double_1'),
    pytest.param(5, 10, id='double_5'),
    pytest.param(0, 0, id='double_0'),
    pytest.param(-3, -6, id='double_negative'),
])
def test_double(input_val, expected):
    assert double(input_val) == expected
```

</details>

### Exercise 3: Test for Errors

Write a parameterized test that checks multiple inputs should raise `ValueError`:

In [ ]:
# Exercise 3: Test error cases

def divide(a, b):
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

# Test that these all raise ValueError:
# divide(1, 0)
# divide(100, 0)
# divide(-5, 0)

# Your parameterized test:
# ...

<details>
<summary><b>Show Answer</b></summary>

```python
@pytest.mark.parametrize("a", [
    pytest.param(1, id='positive'),
    pytest.param(100, id='large'),
    pytest.param(-5, id='negative'),
])
def test_divide_by_zero(a):
    with pytest.raises(ValueError) as e:
        divide(a, 0)
    assert str(e.value) == "Cannot divide by zero"
```

</details>

---

## Running Our Project's Tests

Our project has 71 parameterized tests. Run them:

```bash
cd media_catalogue
pytest -v
```

You'll see output like:

```
tests/test_media_catalogue.py::TestMovie::test_create_valid_movie PASSED
tests/test_media_catalogue.py::TestMovie::test_movie_str_representation PASSED
tests/test_media_catalogue.py::TestMovie::test_valid_movie_boundary_cases[year_1895_first_valid] PASSED
tests/test_media_catalogue.py::TestMovie::test_valid_movie_boundary_cases[duration_1_minimum] PASSED
...
```

Notice how each parameterized case shows its name in brackets!

---

## Key Takeaways

1. **`@pytest.mark.parametrize`** runs one test with multiple inputs

2. **Multiple parameters** - pass tuples of values for each test case

3. **`pytest.param` with `id=`** gives test cases readable names

4. **Boundary tests** - test edge cases like minimum values, maximum values, off-by-one

5. **Type validation** - test what happens with wrong types

6. **Less repetition** - 71 tests in our project come from just a handful of test functions

---

## Next Lessons: PySide6 GUI (Python Files)

You've completed Part 2: Testing!

In Part 3, you'll learn PySide6 and build the GUI:
- What PySide6/Qt is
- Widgets, layouts, and signals
- Building the Media Catalogue interface

---

## IMPORTANT: GUI Lessons are Python Files

**The GUI lessons (08-11) are Python files, NOT Jupyter notebooks.**

PySide6/Qt doesn't work properly in Jupyter, so the GUI lessons are standalone Python files that you run from the terminal.

**To run the GUI lessons:**

```bash
# Lesson 8: PySide6 Introduction
python lessons/08_pyside6_intro.py

# Lesson 9: Building Forms and Tables
python lessons/09_building_gui.py

# Lesson 10: Putting It All Together
python lessons/10_putting_together.py

# Lesson 11: Embedding a Web Browser
python lessons/11_web_browser.py
```

Each file contains:
- Detailed comments explaining the concepts
- Multiple runnable examples you can choose from
- References to YOUR actual code in `main_window.py`

---

## Navigation

| | |
|:---|---:|
| [**← Previous: Lesson 6 - Testing Basics**](06_testing_basics.ipynb) | **Next: Run `python lessons/08_pyside6_intro.py`** |